In [1]:
!pip install opencv-python
!pip install opencv-python-headless
!pip install openvino
!pip install openvino-dev
!pip install openvino-telemetry
!pip install supervision
!pip install ultralytics
!pip install vidgear
!pip install yt-dlp

In [1]:

# import
from vidgear.gears import CamGear
import cv2
import time

source="https://youtu.be/z545k7Tcb5o"
# YouTube Video URL as input source
# Stream Mode (`stream_mode = True`)
stream = CamGear(
    source=source, stream_mode=True, logging=True,  time_delay=0
).start()
video_metadata=stream.ytv_metadata

print(video_metadata.keys())

print(video_metadata['fps'])
print(video_metadata['format'])
print(video_metadata['format_index'])

# search available resolution
resolutions=[format['resolution'] for format in video_metadata['formats']]
for res in resolutions:
    print(res)

# select the desired resolution to get right url
desired_resolution = '1280x720'
for format in video_metadata['formats']:

    if format['resolution'] == desired_resolution:
        VIDEO = format['url']
        break

print(VIDEO)


C:\Users\HP\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,
11:39:10 ::    Helper     ::   INFO   :: Running VidGear Version: 0.3.3
11:39:10 ::    Helper     ::  DEBUG   :: Selecting `best` resolution for streams.
11:39:10 ::    CamGear    ::   INFO   :: Verifying Streaming URL using yt-dlp backend. Please wait...
11:39:14 ::    CamGear    ::   INFO   :: [Backend] :: Streaming URL is fully supported. Available Streams are: [144p, 240p, 360p, 480p, 720p, best, worst]
11:39:14 ::    CamGear    :: WARNING  :: Livestream URL detected. It is strongly recommended to use the GStreamer backend (`backend=cv2.CAP_GSTREAMER`) with these URLs.
11:39:14 ::    CamGear    ::  DEBUG   :: Using `best` resolution for streaming.
11:39:14 ::    CamGear    ::  DEBUG   :: YouTube source ID: `z545k7Tcb5o`, Title: `Périphérique Nord - Porte de la Pape 2024-08-15 11:39`, Q

dict_keys(['id', 'title', 'formats', 'thumbnails', 'thumbnail', 'description', 'channel_id', 'channel_url', 'view_count', 'average_rating', 'age_limit', 'webpage_url', 'categories', 'tags', 'playable_in_embed', 'live_status', 'release_timestamp', '_format_sort_fields', 'automatic_captions', 'subtitles', 'comment_count', 'chapters', 'heatmap', 'like_count', 'concurrent_view_count', 'channel', 'channel_follower_count', 'uploader', 'uploader_id', 'uploader_url', 'upload_date', 'timestamp', 'availability', 'original_url', 'webpage_url_basename', 'webpage_url_domain', 'extractor', 'extractor_key', 'playlist', 'playlist_index', 'display_id', 'fulltitle', 'release_date', 'release_year', 'is_live', 'was_live', 'requested_subtitles', '_has_drm', 'epoch', 'format_id', 'format_index', 'url', 'manifest_url', 'tbr', 'ext', 'fps', 'protocol', 'preference', 'quality', 'has_drm', 'width', 'height', 'vcodec', 'acodec', 'dynamic_range', 'source_preference', 'resolution', 'aspect_ratio', 'http_headers', 

In [20]:
import numpy as np
import supervision as sv
import cv2
import os
from collections import defaultdict, deque
# os.environ.pop("QT_QPA_PLATFORM_PLUGIN_PATH")
# from supervision import draw_text , Color
from ultralytics import YOLO
# load yolo model and get class name
MODEL = "models/yolov8s.pt"
# MODEL = "models/yolov9c.pt"
model=YOLO(MODEL)
CLASS_NAMES_DICT = model.model.names
print(CLASS_NAMES_DICT)
# load openvino model to get faster FPS
#model = YOLO("models/yolov8s_openvino_model/", task='detect')
# model = YOLO("models/yolov9c_openvino_model/", task='detect')

# model=YOLO(MODEL)
# model.fuse()

colors = sv.ColorPalette.LEGACY

video_info = sv.VideoInfo.from_video_path(VIDEO)
print(video_info)
# calculate ratio between video stream and displayed size (here's 1280)

coef=video_info.width/1280
# print(coef)

# polygon design
#  ----> x
# |         (x4,y4)   (x3,y3)
# |              +-------+
#               +-------+
# y            +-------+
#         (x1,y1)    (x2,y2)

# 3 polygons so 3 values in each coordinate from left to right
#    [zone1,zone2, zone3]
x1 = [-160 , -25 , 971  ]
y1 = [ 405 , 710 , 671  ]
x2 = [ 112 , 568 , 1480 ]
y2 = [ 503 , 710 , 671  ]
x3 = [ 557 , 706 , 874  ]
y3 = [ 195 , 212 , 212  ]
x4 = [ 411 , 569 , 749  ]
y4 = [ 195 , 212 , 212  ]
# transform according video stream and displayed video ratio
x1, y1, x2, y2, x3, y3, x4, y4 = map(lambda x: [value * coef for value in x], [x1, y1, x2, y2, x3, y3, x4, y4])


# search middle point of the polygon (x1+x4)/2) or tier point from top ( x1 + 2* x4) / 3) to draw line for counting
x14 = [( x1 + 2 * x4) / 3
       for x1,x4
       in zip(x1,x4)]
y14 = [( y1 + 2 * y4) / 3
       for y1,y4
       in zip(y1,y4)]
x23 = [ ( x2 + 2 * x3) / 3
       for x2,x3
       in zip(x2,x3)]
y23 = [( y2 + 2 * y3) / 3
       for y2,y3
       in zip(y2,y3)]

# polygon zone from left to right (becarefull must be in the same order than le linezone)
polygons = [
  np.array([
 [x1, y1],[x2 , y2],[x3 , y3],[x4 , y4]
  ],np.int32)
 for x1, y1, x2, y2, x3, y3, x4, y4
 in zip(x1, y1, x2, y2, x3, y3, x4, y4)
]


# initialize our zones

zones = [
    sv.PolygonZone(
        polygon = polygon,
        frame_resolution_wh = video_info.resolution_wh
    )
    for polygon
    in polygons
]
zone_annotators = [
    sv.PolygonZoneAnnotator(
        zone = zone,
        color = colors.by_idx(index),
        thickness = 2,
        text_thickness = 1,
        text_scale = 0.5,
    )
    for index, zone
    in enumerate(zones)
]

label_annotators = [
    sv.LabelAnnotator(
        text_position = sv.Position.TOP_CENTER,
        color=colors.by_idx(index),
        text_thickness = 1,
        text_scale = 0.5,
        )
        for index
        in range(len(zones))
]

# box_annotators = [
#     sv.BoxAnnotator(
#         color=colors.by_idx(index),
#         thickness=1,
#         text_thickness=1,
#         text_scale=0.5
#         )
#     for index
#     in range(len(polygons))
# ]
box_annotators = [
    sv.BoundingBoxAnnotator(
        color = colors.by_idx(index),
        thickness = 1,
        )
    for index
    in range(len(polygons))
]

trace_annotators=[
    sv.TraceAnnotator(
        color = colors.by_idx(index),
        thickness = 1,
        trace_length = video_info.fps * 1.5,
        position = sv.Position.BOTTOM_CENTER,
        )
    for index
    in range(len(polygons))
]


lines_start=[

    sv.Point(x14, y14)
    for x14,y14
    in zip(x14,y14)

]

lines_end =[

    sv.Point(x23, y23)
    for x23,y23
    in zip(x23,y23)
]

positions=[(sv.Position.CENTER,sv.Position.CENTER),
           (sv.Position.CENTER,sv.Position.CENTER),
           (sv.Position.CENTER,sv.Position.CENTER),
          ]

line_zones = [ sv.LineZone(start=line_start, end=line_end, triggering_anchors=position)
            for line_start, line_end, position
            in zip(lines_start,lines_end,positions)
]

# for automatic line zone annotator not use here want to use a custom one
line_zone_annotators = [sv.LineZoneAnnotator(thickness = 1,
                                           color = colors.by_idx(index),
                                            text_thickness = 1,
                                              text_scale = 0.5,
                                                text_offset = 4)
    for index
    in range(len(line_zones))
]

# couting line zone text position
text_pos=[ sv.Point (x = 100,y = 320),
            sv.Point (x = 700,y = 320),
            sv.Point (x = 1077,y = 320)

]
byte_tracker = sv.ByteTrack(track_thresh=0.25, track_buffer=100, match_thresh=0.8, frame_rate=video_info.fps)

# byte_tracker = sv.ByteTrack()
fps_monitor = sv.FPSMonitor()
heat_map = sv.HeatMapAnnotator ()
smoother = sv.DetectionsSmoother()




{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microw

VideoInfo(width=1280, height=720, fps=30, total_frames=-3074457345618259)


SupervisionWarnings: The `frame_resolution_wh` parameter is no longer required and will be dropped in version supervision-0.24.0. The mask resolution is now calculated automatically based on the polygon coordinates.
SupervisionWarnings: The `frame_resolution_wh` parameter is no longer required and will be dropped in version supervision-0.24.0. The mask resolution is now calculated automatically based on the polygon coordinates.
SupervisionWarnings: The `frame_resolution_wh` parameter is no longer required and will be dropped in version supervision-0.24.0. The mask resolution is now calculated automatically based on the polygon coordinates.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator` will be removed in supervision-0.26.0.
SupervisionWarnings: BoundingBoxAnnotator is deprecated: `BoundingBoxAnnotator` is deprecated and has been renamed to `BoxAnnotator`. `BoundingBoxAnnotator`

In [21]:
SOURCES = np.array([[
    [x4[0], y4[0]],
    [x3[0], y3[0]],
    [x2[0], y2[0]],
    [x1[0], y1[0]]

],[ [x4[1], y4[1]],
    [x3[1], y3[1]],
    [x2[1], y2[1]],
    [x1[1], y1[1]]
],

[
    [x4[2], y4[2]],
    [x3[2], y3[2]],
    [x2[2], y2[2]],
    [x1[2], y1[2]]
]])

#zone1 in meters
TARGET_WIDTH = 6
TARGET_HEIGHT = 75

TARGETS = np.array([
    [0, 0],
    [TARGET_WIDTH - 1, 0],
    [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
    [0, TARGET_HEIGHT - 1],
])

#zone 2 in meters
TARGET_WIDTH = 6
TARGET_HEIGHT = 85

TARGETS= np.append(TARGETS, np.array([
    [0, 0],
    [TARGET_WIDTH - 1, 0],
    [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
    [0, TARGET_HEIGHT - 1],
]), axis=0)

#zone3 in meters
TARGET_WIDTH = 6
TARGET_HEIGHT = 80


TARGETS = np.append(TARGETS, np.array([
    [0, 0],
    [TARGET_WIDTH - 1, 0],
    [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
    [0, TARGET_HEIGHT - 1],
]),axis=0)

TARGETS = TARGETS.reshape(3, 4, 2)



class ViewTransformer:
    def __init__(self, source: np.ndarray, target: np.ndarray) -> None:
        source = source.astype(np.float32)
        target = target.astype(np.float32)
        self.m = cv2.getPerspectiveTransform(source, target)

    def transform_points(self, points: np.ndarray) -> np.ndarray:
        if points.size == 0:
            return points

        reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
        transformed_points = cv2.perspectiveTransform(
                reshaped_points, self.m)
        return transformed_points.reshape(-1, 2)

# create the transformers matrix for each zone
view_transformers=[ViewTransformer(source=s, target=t)
                  for s,t
                  in zip(SOURCES, TARGETS)]


In [22]:
labels = []

selected_classes = [2, 3, 5, 7] # car, motorcycle, bus, truck from coco classes
#the dictionary that we will use to store the coordinates for each zone
coordinates = defaultdict(lambda: deque(maxlen=30))
coordinates = np.append(coordinates,defaultdict(lambda: deque(maxlen=30)))
coordinates = np.append(coordinates,defaultdict(lambda: deque(maxlen=30)))

def process_frame(frame: np.ndarray, fps) -> np.ndarray:
    speed_labels = [],[],[]

    results = model(frame, imgsz=640, verbose=False)[0]
    # results = model(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = detections[np.isin(detections.class_id, selected_classes)] # filer on selected classes
    detections = byte_tracker.update_with_detections(detections)
    # detections = smoother.update_with_detections(detections)

    # copy frame before annotate
    annotated_frame = frame.copy()

    for i, (zone, zone_annotator, box_annotator, trace_annotator, line_zone,
            line_zone_annotator,
            label_annotator,
            line_start,
            line_end,
            view_transformer,
            speed_label,coordinate
            ) in  enumerate(zip(zones, zone_annotators, box_annotators,
                                trace_annotators,
                                line_zones,
                                line_zone_annotators,
                                label_annotators,
                                lines_start,
                                lines_end,
                                view_transformers,
                                speed_labels,
                                coordinates
                                )
                            ):

        mask = zone.trigger(detections=detections)
        detections_filtered = detections[mask]

        points = detections_filtered.get_anchors_coordinates(
                anchor=sv.Position.BOTTOM_CENTER)

        # plug the view transformer into an existing detection pipeline

        points = view_transformer.transform_points(points=points).astype(int)

        for tracker_id, [_, y] in zip(detections_filtered.tracker_id, points):
            coordinate[tracker_id].append(y)

        # wait to have enough data
        for tracker_id in detections_filtered.tracker_id:
                        if len(coordinate[tracker_id]) < fps/2:
                            # print(coordinates[tracker_id], " - id :", tracker_id, 'len : ', len(coordinates[tracker_id]))
                            speed_label.append(f"#{tracker_id}")

                        else:
                            try:
                                coordinate_start = coordinate[tracker_id][-1]
                                coordinate_end = coordinate[tracker_id][0]
                                distance = abs(coordinate_start - coordinate_end)
                                time = len(coordinate[tracker_id]) / fps
                                speed = distance / time * 3.6
                                speed_label.append(f"{int(speed)} km/h")

                            except:

                                speed_label.append(f"#{tracker_id}")

                                pass
        # labels = [
        # f"#{tracker_id} "
        # for _,_,_,_,tracker_id in detections_filtered]
        # line_zone.trigger(detections=detections_filtered)
        annotated_frame = sv.draw_line(scene=annotated_frame, start=line_start, end=line_end, color=colors.by_idx(i) )
        # annotated_frame = zone_annotator.annotate(scene=annotated_frame, label=f"Dir. Ouest : {i+random.randint(0,100)}")

        annotated_frame = zone_annotator.annotate(
            scene=annotated_frame,
            label=f"Dir. Ouest : {line_zone.in_count}") if i==0 else zone_annotator.annotate(
                scene=annotated_frame, label=f"Dir. Est : {line_zone.out_count}")

        annotated_frame = label_annotator.annotate(scene=annotated_frame,
                                                  detections=detections_filtered,
                                                  labels=speed_label)

        # annotated_frame=line_zone_annotator.annotate(annotated_frame,line_counter=line_zone )
        annotated_frame = box_annotator.annotate(scene=annotated_frame,
                                                  detections=detections_filtered,
                                                  )

        annotated_frame = trace_annotator.annotate(scene=annotated_frame,detections=detections_filtered )
        line_zone.trigger(detections=detections_filtered)
        # print(line_zone.in_count)
        # print(line_zone.out_count)


    return annotated_frame

In [23]:

'''coordinates = [defaultdict(lambda: deque(maxlen=30)) for _ in range(3)]

def process_frame(frame: np.ndarray, fps: int) -> np.ndarray:
    # List to store speed labels for each zone
    speed_labels = [[] for _ in range(3)]

    # Get detections from the model
    results = model(frame, imgsz=640, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = detections[np.isin(detections.class_id, selected_classes)]

    # Update detections using the tracker
    detections = byte_tracker.update_with_detections(detections)

    # Get the anchor points for filtered detections
    points = detections.get_anchors_coordinates(anchor=sv.Position.BOTTOM_CENTER)

    # Initialize annotated_frame with the original frame
    annotated_frame = frame.copy()

    # Iterate through zones and corresponding annotators
    for i, (zone, zone_annotator, box_annotator, trace_annotator, line_zone,
            line_zone_annotator, label_annotator, line_start, line_end,
            view_transformer, speed_label, coordinate) in enumerate(zip(
            zones, zone_annotators, box_annotators, trace_annotators,
            line_zones, line_zone_annotators, label_annotators, lines_start,
            lines_end, view_transformers, speed_labels, coordinates)):

        # Trigger zone and filter detections
        mask = zone.trigger(detections=detections)
        detections_filtered = detections[mask]

        # Transform points and update coordinates
        points_transformed = view_transformer.transform_points(points=points[mask]).astype(int)
        for tracker_id, [_, y] in zip(detections_filtered.tracker_id, points_transformed):
            coordinate[tracker_id].append(y)

        # Calculate speed for each tracked object
        for tracker_id in detections_filtered.tracker_id:
            if len(coordinate[tracker_id]) >= fps / 2:
                try:
                    distance = abs(coordinate[tracker_id][-1] - coordinate[tracker_id][0])
                    time = len(coordinate[tracker_id]) / fps
                    speed = distance / time * 3.6
                    speed_label.append(f"{int(speed)} km/h")
                except Exception:
                    speed_label.append(f"#{tracker_id}")
            else:
                speed_label.append(f"#{tracker_id}")

        # Annotate the frame
        annotated_frame = sv.draw_line(annotated_frame, start=line_start, end=line_end, color=colors.by_idx(i))
        annotated_frame = zone_annotator.annotate(
            scene=annotated_frame,
            label=f"Dir. Ouest : {line_zone.in_count}" if i == 0 else f"Dir. Est : {line_zone.out_count}"
        )
        annotated_frame = label_annotator.annotate(annotated_frame, detections=detections_filtered, labels=speed_label)
        annotated_frame = box_annotator.annotate(annotated_frame, detections=detections_filtered)
        annotated_frame = trace_annotator.annotate(annotated_frame, detections=detections_filtered)

        # Trigger the line zone for counting
        line_zone.trigger(detections=detections_filtered)

    return annotated_frame'''


'coordinates = [defaultdict(lambda: deque(maxlen=30)) for _ in range(3)]\n\ndef process_frame(frame: np.ndarray, fps: int) -> np.ndarray:\n    # List to store speed labels for each zone\n    speed_labels = [[] for _ in range(3)]\n\n    # Get detections from the model\n    results = model(frame, imgsz=640, verbose=False)[0]\n    detections = sv.Detections.from_ultralytics(results)\n    detections = detections[np.isin(detections.class_id, selected_classes)]\n\n    # Update detections using the tracker\n    detections = byte_tracker.update_with_detections(detections)\n\n    # Get the anchor points for filtered detections\n    points = detections.get_anchors_coordinates(anchor=sv.Position.BOTTOM_CENTER)\n\n    # Initialize annotated_frame with the original frame\n    annotated_frame = frame.copy()\n\n    # Iterate through zones and corresponding annotators\n    for i, (zone, zone_annotator, box_annotator, trace_annotator, line_zone,\n            line_zone_annotator, label_annotator, line_s

In [31]:
# for direct show
cap = cv2.VideoCapture(VIDEO)
#fps = int(cap.get(cv2.CAP_PROP_FPS))
fps=video_info.fps
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"FPS: {fps}")
print(f"image : {width}x{height}")

while True:
    ret, frame = cap.read()
    if not ret:
        break
    # frame=cv2.resize(frame,(1280,720))
    show=process_frame(frame,int(fps))

    fps_monitor.tick()
    #fps = fps_monitor()
    fps_text = f"FPS: {fps:.0f}"
    cv2.putText(show, fps_text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Counting", show)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()


FPS: 30
image : 1280x720


In [ ]:
# to save video instead of displaying 
side=0
output_file = 'output_video.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  #for mp4
out = cv2.VideoWriter(output_file, fourcc, video_info.fps, (video_info.width, video_info.height))

cap = cv2.VideoCapture(VIDEO)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(f"FPS: {fps}")
print(f"image : {width}x{height}")
# start time of the recording
start_time = time.time()
# duration of the recording
duration = 120

while (time.time() - start_time) < duration:
    ret, frame = cap.read()
    if not ret:
        break
    # frame=cv2.resize(frame,(1280,720))
    show=process_frame(frame,0,fps)
    fps_monitor.tick()
    fps = fps_monitor()
    fps_text = f"FPS: {fps:.0f}"
    cv2.putText(show, fps_text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # cv2.imshow("Counting", show)
    out.write(show)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
out.release()
cap.release()
cv2.destroyAllWindows()


**Distance**

In [ ]:
from vidgear.gears import CamGear
import cv2
import numpy as np
import supervision as sv
from collections import defaultdict, deque
from ultralytics import YOLO
import time


def main():

    def video_manifest_extractor(source):
        """
        Function to extract metadata from a YouTube video source
          and find the desired resolution URL.

        Parameters:
        source (str): Video source URL (ex. "<https://youtu.be/bvetuLwJIkA>")

        Returns:
        str: Desired resolution video URL
        """
        stream = CamGear(source=source, stream_mode=True, logging=True,
                         time_delay=0).start()
        video_metadata = stream.ytv_metadata

        print(video_metadata.keys())
        print(video_metadata["fps"])
        print(video_metadata["format"])
        print(video_metadata["format_index"])

        resolutions = [format["resolution"] for format in video_metadata["formats"]]
        for res in resolutions:
            print(res)

        resolution_desiree = '1280x720'
        for format in video_metadata["formats"]:
            if format["resolution"] == resolution_desiree:
                VIDEO = format["url"]
                return VIDEO

    source = "https://youtu.be/z545k7Tcb5o"
    VIDEO = video_manifest_extractor(source)
    print(VIDEO)

    # Load OpenVINO model 
    MODEL = "models/yolov8s.pt"
    model = YOLO(MODEL)

    # Get class names dictionary
    CLASS_NAMES_DICT = model.model.names
    print(CLASS_NAMES_DICT)

    model_openvino = YOLO("models/yolov8s_openvino_model/", task="detect")
    colors = sv.ColorPalette.LEGACY

    video_info = sv.VideoInfo.from_video_path(VIDEO)

    # Calculate the scaling coefficient based on the video width and
    #  the desired output width (1280)
    coef = video_info.width / 1280
    # print(coef)

    # polygon design
    #  ----> x
    # |         (x4,y4)   (x3,y3)
    # |              +-------+
    #               +-------+
    # y            +-------+
    #         (x1,y1)    (x2,y2)

    # 3 polygons so 3 values in each coordinate from left to right 
    #    [zone1,zone2, zone3]
    x1 = [-160, -25, 971]
    y1 = [405, 710, 671]
    x2 = [112, 568, 1480]
    y2 = [503, 710, 671]
    x3 = [557, 706, 874]
    y3 = [195, 212, 212]
    x4 = [411, 569, 749]
    y4 = [195, 212, 212]

    # Scale coordinates according to the video flow and the 
    # aspect ratio of the displayed video
    x1, y1, x2, y2, x3, y3, x4, y4 = map(
        lambda vals: list(map(lambda val: val * coef, vals)),
        [x1, y1, x2, y2, x3, y3, x4, y4]
    )

    # Find the centroid or third point from top of the polygon
    # e.g.: ((x1 + 2* x4) / 3) for drawing counting lines
    x14 = list(map(lambda x1, x4: (x1 + 2 * x4) / 3, x1, x4))
    y14 = list(map(lambda y1, y4: (y1 + 2 * y4) / 3, y1, y4))
    x23 = list(map(lambda x2, x3: (x2 + 2 * x3) / 3, x2, x3))
    y23 = list(map(lambda y2, y3: (y2 + 2 * y3) / 3, y2, y3))

    # Polygon zones defined from left to right (the same order
    #  as the linezone)
    polygons = [
        np.array([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], dtype=np.int32)
        for x1, y1, x2, y2, x3, y3, x4, y4
        in zip(x1, y1, x2, y2, x3, y3, x4, y4)
    ]
    # initialize our zones
    zones = [
        sv.PolygonZone(
            polygon=polygon,
            frame_resolution_wh=video_info.resolution_wh
        )
        for polygon
        in polygons
    ]
    zone_annotators = [
        sv.PolygonZoneAnnotator(
            zone=zone,
            color=colors.by_idx(index),
            thickness=2,
            text_thickness=1,
            text_scale=0.5,
        )
        for index, zone
        in enumerate(zones)
    ]
    label_annotators = [
        sv.LabelAnnotator(
            text_position=sv.Position.TOP_CENTER,
            color=colors.by_idx(index),
            text_thickness=1,
            text_scale=0.5,
        )
        for index
        in range(len(zones))
    ]
    # box_annotators = [
    #     sv.BoxAnnotator(
    #         color=colors.by_idx(index),
    #         thickness=1,
    #         text_thickness=1,
    #         text_scale=0.5
    #         )
    #     for index
    #     in range(len(polygons))
    # ]
    box_annotators = [
        sv.BoundingBoxAnnotator(
            color=colors.by_idx(index),
            thickness=1,
            )
        for index
        in range(len(polygons))
    ]

    trace_annotators = [
        sv.TraceAnnotator(
            color=colors.by_idx(index),
            thickness=1,
            trace_length=video_info.fps * 1.5,
            position=sv.Position.BOTTOM_CENTER,
            )
        for index
        in range(len(polygons))
    ]
    lines_start = [
        sv.Point(x14, y14)
        for x14, y14
        in zip(x14, y14)
    ]
    lines_end = [
        sv.Point(x23, y23)
        for x23, y23
        in zip(x23, y23)
    ]
    positions = [
        (sv.Position.CENTER, sv.Position.CENTER),
        (sv.Position.CENTER, sv.Position.CENTER),
        (sv.Position.CENTER, sv.Position.CENTER),
    ]
    line_zones = [
        sv.LineZone(start=line_start, end=line_end,
                    triggering_anchors=position)
        for line_start, line_end, position
        in zip(lines_start, lines_end, positions)
    ]
    # for automatic line zone annotator not use here want to use a custom one
    line_zone_annotators = [
        sv.LineZoneAnnotator(thickness=1,
                             color=colors.by_idx(index),
                             text_thickness=1,
                             text_scale=0.5,
                             text_offset=4)
        for index
        in range(len(line_zones))
    ]
    # couting line zone text position
    # text_pos = [
    #     sv.Point(x=100, y=320),
    #     sv.Point(x=700, y=320),
    #     sv.Point(x=1077, y=320)
    # ]
    # initialyze ByteTracker
    byte_tracker = sv.ByteTrack(
        track_thresh=0.25,
        track_buffer=100, 
        match_thresh=0.8,
        frame_rate=video_info.fps
    )
    # byte_tracker = sv.ByteTrack()
    fps_monitor = sv.FPSMonitor()
    # heat_map = sv.HeatMapAnnotator()
    # smoother = sv.DetectionsSmoother()
    # intialize the source coordinate for speed estimation
    SOURCES = np.array([[
        [x4[0], y4[0]],
        [x3[0], y3[0]],
        [x2[0], y2[0]],
        [x1[0], y1[0]]

    ], [[x4[1], y4[1]],
        [x3[1], y3[1]],
        [x2[1], y2[1]],
        [x1[1], y1[1]]],
          [[x4[2], y4[2]],
           [x3[2], y3[2]],
           [x2[2], y2[2]],
           [x1[2], y1[2]]
           ]
           ])
    # initialize Target real(in meters) coordinate 
    # zone1 in meters
    TARGET_WIDTH = 6
    TARGET_HEIGHT = 75
    TARGETS = np.array([
        [0, 0],
        [TARGET_WIDTH - 1, 0],
        [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
        [0, TARGET_HEIGHT - 1],
    ])
    # zone 2 in meters
    TARGET_WIDTH = 6
    TARGET_HEIGHT = 85
    TARGETS = np.append(TARGETS, np.array([
        [0, 0],
        [TARGET_WIDTH - 1, 0],
        [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
        [0, TARGET_HEIGHT - 1],
    ]), axis=0)
    # zone3 in meters
    TARGET_WIDTH = 6
    TARGET_HEIGHT = 80
    TARGETS = np.append(TARGETS, np.array([
        [0, 0],
        [TARGET_WIDTH - 1, 0],
        [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
        [0, TARGET_HEIGHT - 1],
    ]), axis=0)

    TARGETS = TARGETS.reshape(3, 4, 2)
    # class searching transformation matrix between
    #  SOURCE and TARGET to get speed estimation

    class ViewTransformer:

        def __init__(self, source: np.ndarray, target: np.ndarray) -> None:
            source = source.astype(np.float32)
            target = target.astype(np.float32)
            self.m = cv2.getPerspectiveTransform(source, target)

        def transform_points(self, points: np.ndarray) -> np.ndarray:
            if points.size == 0:
                return points

            reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
            transformed_points = cv2.perspectiveTransform(
                    reshaped_points, self.m)
            return transformed_points.reshape(-1, 2)

    # create the transformers matrix for each zone
    view_transformers = [
        ViewTransformer(source=s, target=t)
        for s, t
        in zip(SOURCES, TARGETS)
    ]
    # car, motorcycle, bus, truck from coco classes
    selected_classes = [2, 3, 5, 7] 
    # initialize the dictionary that we 
    # will use to store the coordinates for each zone
    coordinates = defaultdict(lambda: deque(maxlen=30))
    coordinates = defaultdict(lambda: deque(maxlen=30))
    coordinates = np.append(coordinates, defaultdict(lambda: deque(maxlen=30)))
    coordinates = np.append(coordinates, defaultdict(lambda: deque(maxlen=30)))
    # frame processing

    '''def process_frame(frame: np.ndarray, fps) -> np.ndarray:
        speed_labels = [], [], []
        zone_car_counts = [0]*len(zones)  # Initialize car counts for each zone
        warning_message = ""

        results = model_openvino(frame, imgsz=640, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = detections[np.isin(detections.class_id, selected_classes)]  # Filter on selected classes
        detections = byte_tracker.update_with_detections(detections)

    # Copy frame before annotating                     
        annotated_frame = frame.copy()

        for i, (zone,
            zone_annotator,
            box_annotator,
            trace_annotator,
            line_zone,
            line_zone_annotator,
            label_annotator,
            line_start,
            line_end,
            view_transformer,
            speed_label,
            coordinate) in enumerate(zip(
                zones,
                zone_annotators,
                box_annotators,
                trace_annotators,
                line_zones,
                line_zone_annotators,
                label_annotators,
                lines_start,
                lines_end,
                view_transformers,
                speed_labels,
                coordinates)):

            mask = zone.trigger(detections=detections)
            detections_filtered = detections[mask]
            zone_car_counts[i] = len(detections_filtered)  # Count cars in this zone

            points = detections_filtered.get_anchors_coordinates(anchor=sv.Position.BOTTOM_CENTER)
            points = view_transformer.transform_points(points=points).astype(int)

        # Calculate distances between cars and check for closeness
            for j, (point1, tracker_id1) in enumerate(zip(points, detections_filtered.tracker_id)):
                for k, (point2, tracker_id2) in enumerate(zip(points, detections_filtered.tracker_id)):
                    if j != k:
                        distance = np.linalg.norm(point1 - point2)
                        if distance < 50:  # Threshold for being "too close"
                            warning_message += f"Cars {tracker_id1} and {tracker_id2} too close in zone {i+1}: {distance:.2f} pixels\n"

            for tracker_id, [_, y] in zip(detections_filtered.tracker_id, points):
                coordinate[tracker_id].append(y)

        # Calculate speed and generate labels
            for tracker_id in detections_filtered.tracker_id:
                if len(coordinate[tracker_id]) < fps/2:
                    speed_label.append(f"#{tracker_id}")
                else:
                    try:
                        coordinate_start = coordinate[tracker_id][-1]
                        coordinate_end = coordinate[tracker_id][0]
                        distance = abs(coordinate_start - coordinate_end)
                        time = len(coordinate[tracker_id]) / fps
                        speed = distance / time * 3.6
                        speed_label.append(f"{int(speed)} km/h")
                    except Exception as e:
                        speed_label.append(f"#{tracker_id}")
                        print(f"An error occurred: {e}")

        # Annotate frame
            direction_label = "Dir. West" if i == 0 else "Dir. East"
            annotated_frame = zone_annotator.annotate(
                scene=annotated_frame,
                label=f"{direction_label} : {line_zone.in_count}") if i == 0 else zone_annotator.annotate(
                scene=annotated_frame,
                label=f"{direction_label} : {line_zone.out_count}")

            annotated_frame = label_annotator.annotate(
                scene=annotated_frame,
                detections=detections_filtered,
                labels=speed_label)

            annotated_frame = box_annotator.annotate(
                scene=annotated_frame,
                detections=detections_filtered)

            annotated_frame = trace_annotator.annotate(
                scene=annotated_frame,
                detections=detections_filtered)

            annotated_frame = sv.draw_line(scene=annotated_frame,
                                           start=line_start,
                                       end=line_end,
                                       color=colors.by_idx(i))

            line_zone.trigger(detections=detections_filtered)

    # Display total car count and warnings on the top of the screen
        total_car_count = sum(zone_car_counts)
        cv2.putText(annotated_frame, f"Total Cars: {total_car_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
        if warning_message:
            cv2.putText(annotated_frame, "WARNING: Cars too close!", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            print(warning_message)  # Also print the warning message in the backend

        return annotated_frame'''
    def process_frame(frame: np.ndarray, fps) -> np.ndarray:
        speed_labels = [], [], []
        zone_car_counts = [0] * len(zones)  # Initialize car counts for each zone
        warning_message = ""

        results = model_openvino(frame, imgsz=640, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = detections[np.isin(detections.class_id, selected_classes)]  # Filter on selected classes
        detections = byte_tracker.update_with_detections(detections)

    # Copy frame before annotating                     
        annotated_frame = frame.copy()

        for i, (zone,
            zone_annotator,
            box_annotator,
            trace_annotator,
            line_zone,
            line_zone_annotator,
            label_annotator,
            line_start,
            line_end,
            view_transformer,
            speed_label,
            coordinate) in enumerate(zip(
                zones,
                zone_annotators,
                box_annotators,
                trace_annotators,
                line_zones,
                line_zone_annotators,
                label_annotators,
                lines_start,
                lines_end,
                view_transformers,
                speed_labels,
                coordinates)):

            mask = zone.trigger(detections=detections)
            detections_filtered = detections[mask]
            zone_car_counts[i] = len(detections_filtered)  # Count cars in this zone

            points = detections_filtered.get_anchors_coordinates(anchor=sv.Position.BOTTOM_CENTER)
            points = view_transformer.transform_points(points=points).astype(int)

        # Calculate distances between cars and check for closeness
            for j, (point1, tracker_id1) in enumerate(zip(points, detections_filtered.tracker_id)):
                for k, (point2, tracker_id2) in enumerate(zip(points, detections_filtered.tracker_id)):
                    if j != k:
                        distance = np.linalg.norm(point1 - point2)
                        if distance < 50:  # Threshold for being "too close"
                            warning_message += f"Cars {tracker_id1} and {tracker_id2} too close in zone {i+1}: {distance:.2f} pixels\n"

            for tracker_id, [_, y] in zip(detections_filtered.tracker_id, points):
                coordinate[tracker_id].append(y)

        # Calculate speed and generate labels
            for tracker_id in detections_filtered.tracker_id:
                if len(coordinate[tracker_id]) < fps/2:
                    speed_label.append(f"#{tracker_id}")
                else:
                    try:
                        coordinate_start = coordinate[tracker_id][-1]
                        coordinate_end = coordinate[tracker_id][0]
                        distance = abs(coordinate_start - coordinate_end)
                        time = len(coordinate[tracker_id]) / fps
                        speed = distance / time * 3.6
                        speed_label.append(f"{int(speed)} km/h")
                    except Exception as e:
                        speed_label.append(f"#{tracker_id}")
                        print(f"An error occurred: {e}")

        # Annotate frame
            direction_label = "Dir. West" if i == 0 else "Dir. East"
            annotated_frame = zone_annotator.annotate(
                scene=annotated_frame,
                label=f"{direction_label} : {line_zone.in_count}") if i == 0 else zone_annotator.annotate(
                scene=annotated_frame,
                label=f"{direction_label} : {line_zone.out_count}")

            annotated_frame = label_annotator.annotate(
                scene=annotated_frame,
                detections=detections_filtered,
                labels=speed_label)

            annotated_frame = box_annotator.annotate(
                scene=annotated_frame,
                detections=detections_filtered)

            annotated_frame = trace_annotator.annotate(
                scene=annotated_frame,
                detections=detections_filtered)

            annotated_frame = sv.draw_line(scene=annotated_frame,
                                       start=line_start,
                                       end=line_end,
                                       color=colors.by_idx(i))

            line_zone.trigger(detections=detections_filtered)

    # Display total car count and warnings in the center of the screen
        total_car_count = sum(zone_car_counts)
        height, width, _ = annotated_frame.shape
        text_position = (width - 300, 50)
        warning_text_position = (width - 500, 100)
    
        cv2.putText(annotated_frame, f"Total Cars: {total_car_count}", text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
    
        if warning_message:
            cv2.putText(annotated_frame, "WARNING: Cars too close!", warning_text_position, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            print(warning_message)  # Also print the warning message in the backend

        return annotated_frame


    # for direct show
    cap = cv2.VideoCapture(VIDEO)
    # fps = int(cap.get(cv2.CAP_PROP_FPS))
    fps = video_info.fps
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"FPS: {fps}")
    print(f"image : {width}x{height}")

    while True:
        start_time = time.time()
        ret, frame = cap.read()
        if not ret:
            break
        # frame=cv2.resize(frame,(1280,720))
        show = process_frame(frame, int(fps))
        fps_monitor.tick()
        #fps = fps_monitor()
        fps_text = f"FPS: {fps:.0f}"
        cv2.putText(show, fps_text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        cv2.imshow("Counting - Speed Estimation", show)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
        time.sleep(max(1/25 - (time.time() - start_time), 0))
    cap.release()
    cv2.destroyAllWindows()


if __name__ == '__main__':
    main()


***MORE FEATURES***

In [ ]:
from vidgear.gears import CamGear
import cv2
import numpy as np
import supervision as sv
from collections import defaultdict, deque
from ultralytics import YOLO
import time


def main():

    def video_manifest_extractor(source):
        """
        Function to extract metadata from a YouTube video source
          and find the desired resolution URL.

        Parameters:
        source (str): Video source URL (ex. "<https://youtu.be/bvetuLwJIkA>")

        Returns:
        str: Desired resolution video URL
        """
        stream = CamGear(source=source, stream_mode=True, logging=True,
                         time_delay=0).start()
        video_metadata = stream.ytv_metadata

        print(video_metadata.keys())
        print(video_metadata["fps"])
        print(video_metadata["format"])
        print(video_metadata["format_index"])

        resolutions = [format["resolution"] for format in video_metadata["formats"]]
        for res in resolutions:
            print(res)

        resolution_desiree = '1280x720'
        for format in video_metadata["formats"]:
            if format["resolution"] == resolution_desiree:
                VIDEO = format["url"]
                return VIDEO

    source = "https://youtu.be/z545k7Tcb5o"
    VIDEO = video_manifest_extractor(source)
    print(VIDEO)

    # Load OpenVINO model 
    MODEL = "models/yolov8s.pt"
    model = YOLO(MODEL)

    # Get class names dictionary
    CLASS_NAMES_DICT = model.model.names
    print(CLASS_NAMES_DICT)

    model_openvino = YOLO("models/yolov8s_openvino_model/", task="detect")
    colors = sv.ColorPalette.LEGACY

    video_info = sv.VideoInfo.from_video_path(VIDEO)

    # Calculate the scaling coefficient based on the video width and
    #  the desired output width (1280)
    coef = video_info.width / 1280
    # print(coef)

    # polygon design
    #  ----> x
    # |         (x4,y4)   (x3,y3)
    # |              +-------+
    #               +-------+
    # y            +-------+
    #         (x1,y1)    (x2,y2)

    # 3 polygons so 3 values in each coordinate from left to right 
    #    [zone1,zone2, zone3]
    x1 = [-160, -25, 971]
    y1 = [405, 710, 671]
    x2 = [112, 568, 1480]
    y2 = [503, 710, 671]
    x3 = [557, 706, 874]
    y3 = [195, 212, 212]
    x4 = [411, 569, 749]
    y4 = [195, 212, 212]

    # Scale coordinates according to the video flow and the 
    # aspect ratio of the displayed video
    x1, y1, x2, y2, x3, y3, x4, y4 = map(
        lambda vals: list(map(lambda val: val * coef, vals)),
        [x1, y1, x2, y2, x3, y3, x4, y4]
    )

    # Find the centroid or third point from top of the polygon
    # e.g.: ((x1 + 2* x4) / 3) for drawing counting lines
    x14 = list(map(lambda x1, x4: (x1 + 2 * x4) / 3, x1, x4))
    y14 = list(map(lambda y1, y4: (y1 + 2 * y4) / 3, y1, y4))
    x23 = list(map(lambda x2, x3: (x2 + 2 * x3) / 3, x2, x3))
    y23 = list(map(lambda y2, y3: (y2 + 2 * y3) / 3, y2, y3))

    # Polygon zones defined from left to right (the same order
    #  as the linezone)
    polygons = [
        np.array([[x1, y1], [x2, y2], [x3, y3], [x4, y4]], dtype=np.int32)
        for x1, y1, x2, y2, x3, y3, x4, y4
        in zip(x1, y1, x2, y2, x3, y3, x4, y4)
    ]
    # initialize our zones
    zones = [
        sv.PolygonZone(
            polygon=polygon,
            frame_resolution_wh=video_info.resolution_wh
        )
        for polygon
        in polygons
    ]
    zone_annotators = [
        sv.PolygonZoneAnnotator(
            zone=zone,
            color=colors.by_idx(index),
            thickness=2,
            text_thickness=1,
            text_scale=0.5,
        )
        for index, zone
        in enumerate(zones)
    ]
    label_annotators = [
        sv.LabelAnnotator(
            text_position=sv.Position.TOP_CENTER,
            color=colors.by_idx(index),
            text_thickness=1,
            text_scale=0.5,
        )
        for index
        in range(len(zones))
    ]
    # box_annotators = [
    #     sv.BoxAnnotator(
    #         color=colors.by_idx(index),
    #         thickness=1,
    #         text_thickness=1,
    #         text_scale=0.5
    #         )
    #     for index
    #     in range(len(polygons))
    # ]
    box_annotators = [
        sv.BoundingBoxAnnotator(
            color=colors.by_idx(index),
            thickness=1,
            )
        for index
        in range(len(polygons))
    ]

    trace_annotators = [
        sv.TraceAnnotator(
            color=colors.by_idx(index),
            thickness=1,
            trace_length=video_info.fps * 1.5,
            position=sv.Position.BOTTOM_CENTER,
            )
        for index
        in range(len(polygons))
    ]
    lines_start = [
        sv.Point(x14, y14)
        for x14, y14
        in zip(x14, y14)
    ]
    lines_end = [
        sv.Point(x23, y23)
        for x23, y23
        in zip(x23, y23)
    ]
    positions = [
        (sv.Position.CENTER, sv.Position.CENTER),
        (sv.Position.CENTER, sv.Position.CENTER),
        (sv.Position.CENTER, sv.Position.CENTER),
    ]
    line_zones = [
        sv.LineZone(start=line_start, end=line_end,
                    triggering_anchors=position)
        for line_start, line_end, position
        in zip(lines_start, lines_end, positions)
    ]
    # for automatic line zone annotator not use here want to use a custom one
    line_zone_annotators = [
        sv.LineZoneAnnotator(thickness=1,
                             color=colors.by_idx(index),
                             text_thickness=1,
                             text_scale=0.5,
                             text_offset=4)
        for index
        in range(len(line_zones))
    ]
    # couting line zone text position
    # text_pos = [
    #     sv.Point(x=100, y=320),
    #     sv.Point(x=700, y=320),
    #     sv.Point(x=1077, y=320)
    # ]
    # initialyze ByteTracker
    byte_tracker = sv.ByteTrack(
        track_thresh=0.25,
        track_buffer=100, 
        match_thresh=0.8,
        frame_rate=video_info.fps
    )
    # byte_tracker = sv.ByteTrack()
    fps_monitor = sv.FPSMonitor()
    # heat_map = sv.HeatMapAnnotator()
    # smoother = sv.DetectionsSmoother()
    # intialize the source coordinate for speed estimation
    SOURCES = np.array([[
        [x4[0], y4[0]],
        [x3[0], y3[0]],
        [x2[0], y2[0]],
        [x1[0], y1[0]]

    ], [[x4[1], y4[1]],
        [x3[1], y3[1]],
        [x2[1], y2[1]],
        [x1[1], y1[1]]],
          [[x4[2], y4[2]],
           [x3[2], y3[2]],
           [x2[2], y2[2]],
           [x1[2], y1[2]]
           ]
           ])
    # initialize Target real(in meters) coordinate 
    # zone1 in meters
    TARGET_WIDTH = 6
    TARGET_HEIGHT = 75
    TARGETS = np.array([
        [0, 0],
        [TARGET_WIDTH - 1, 0],
        [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
        [0, TARGET_HEIGHT - 1],
    ])
    # zone 2 in meters
    TARGET_WIDTH = 6
    TARGET_HEIGHT = 85
    TARGETS = np.append(TARGETS, np.array([
        [0, 0],
        [TARGET_WIDTH - 1, 0],
        [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
        [0, TARGET_HEIGHT - 1],
    ]), axis=0)
    # zone3 in meters
    TARGET_WIDTH = 6
    TARGET_HEIGHT = 80
    TARGETS = np.append(TARGETS, np.array([
        [0, 0],
        [TARGET_WIDTH - 1, 0],
        [TARGET_WIDTH - 1, TARGET_HEIGHT - 1],
        [0, TARGET_HEIGHT - 1],
    ]), axis=0)

    TARGETS = TARGETS.reshape(3, 4, 2)
    # class searching transformation matrix between
    #  SOURCE and TARGET to get speed estimation

    class ViewTransformer:

        def __init__(self, source: np.ndarray, target: np.ndarray) -> None:
            source = source.astype(np.float32)
            target = target.astype(np.float32)
            self.m = cv2.getPerspectiveTransform(source, target)

        def transform_points(self, points: np.ndarray) -> np.ndarray:
            if points.size == 0:
                return points

            reshaped_points = points.reshape(-1, 1, 2).astype(np.float32)
            transformed_points = cv2.perspectiveTransform(
                    reshaped_points, self.m)
            return transformed_points.reshape(-1, 2)

    # create the transformers matrix for each zone
    view_transformers = [
        ViewTransformer(source=s, target=t)
        for s, t
        in zip(SOURCES, TARGETS)
    ]
    # car, motorcycle, bus, truck from coco classes
    selected_classes = [2, 3, 5, 7] 
    # initialize the dictionary that we 
    # will use to store the coordinates for each zone
    coordinates = defaultdict(lambda: deque(maxlen=30))
    coordinates = defaultdict(lambda: deque(maxlen=30))
    coordinates = np.append(coordinates, defaultdict(lambda: deque(maxlen=30)))
    coordinates = np.append(coordinates, defaultdict(lambda: deque(maxlen=30)))
    # frame processing
    def process_frame(frame: np.ndarray, fps) -> np.ndarray:
        speed_labels = [], [], []
        zone_car_counts = [0] * len(zones)  # Initialize car counts for each zone based on the number of zones
        warning_message = ""
        
        results = model_openvino(frame, imgsz=640, verbose=False)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = detections[np.isin(detections.class_id, selected_classes)]  # Filter on selected classes
        detections = byte_tracker.update_with_detections(detections)
    
        # Copy frame before annotating                     
        annotated_frame = frame.copy()
        height, width, _ = annotated_frame.shape  # Get frame dimensions
    
        # Initialize vehicle counts by type
        vehicle_type_counts = {class_id: 0 for class_id in selected_classes}
    
        for i, (zone,
                zone_annotator,
                box_annotator,
                trace_annotator,
                line_zone,
                line_zone_annotator,
                label_annotator,
                line_start,
                line_end,
                view_transformer,
                speed_label,
                coordinate) in enumerate(zip(
                    zones,
                    zone_annotators,
                    box_annotators,
                    trace_annotators,
                    line_zones,
                    line_zone_annotators,
                    label_annotators,
                    lines_start,
                    lines_end,
                    view_transformers,
                    speed_labels,
                    coordinates)):
    
            mask = zone.trigger(detections=detections)
            detections_filtered = detections[mask]
            zone_car_counts[i] = len(detections_filtered)  # Count cars in this zone
    
            points = detections_filtered.get_anchors_coordinates(anchor=sv.Position.BOTTOM_CENTER)
            points = view_transformer.transform_points(points=points).astype(int)
    
            # Calculate distances between cars 
            for j, (point1, tracker_id1) in enumerate(zip(points, detections_filtered.tracker_id)):
                for k, (point2, tracker_id2) in enumerate(zip(points, detections_filtered.tracker_id)):
                    if j != k:
                        distance = np.linalg.norm(point1 - point2)
                        if distance < 50:  # Threshold for being "too close"
                            warning_message += f"Cars {tracker_id1} and {tracker_id2} too close in zone {i+1}: {distance:.2f} pixels\n"
    
            for tracker_id, [_, y] in zip(detections_filtered.tracker_id, points):
                coordinate[tracker_id].append(y)
    
            # Calculate speed and generate labels
            for tracker_id in detections_filtered.tracker_id:
                if len(coordinate[tracker_id]) < fps/2:
                    speed_label.append(f"#{tracker_id}")
                else:
                    try:
                        coordinate_start = coordinate[tracker_id][-1]
                        coordinate_end = coordinate[tracker_id][0]
                        distance = abs(coordinate_start - coordinate_end)
                        time = len(coordinate[tracker_id]) / fps
                        speed = distance / time * 3.6
                        speed_label.append(f"{int(speed)} km/h")
    
                        # Real-time speed monitoring with alerts
                        if speed > speed_limit:  
                            warning_message += f"Car {tracker_id} in zone {i+1} is overspeeding at {int(speed)} km/h\n"
    
                    except Exception as e:
                        speed_label.append(f"#{tracker_id}")
                        print(f"An error occurred: {e}")
    
            #Update vehicle counts by type
            for class_id in detections_filtered.class_id:
                vehicle_type_counts[class_id] += 1
    
            # Annotate frame
            direction_label = "Dir. West" if i == 0 else "Dir. East"
            annotated_frame = zone_annotator.annotate(
                scene=annotated_frame,
                label=f"{direction_label} : {line_zone.in_count}") if i == 0 else zone_annotator.annotate(
                    scene=annotated_frame,
                    label=f"{direction_label} : {line_zone.out_count}")
    
            annotated_frame = label_annotator.annotate(
                scene=annotated_frame,
                detections=detections_filtered,
                labels=speed_label)
    
            annotated_frame = box_annotator.annotate(
                scene=annotated_frame,
                detections=detections_filtered)
    
            annotated_frame = trace_annotator.annotate(
                scene=annotated_frame,
                detections=detections_filtered)
    
            annotated_frame = sv.draw_line(scene=annotated_frame,
                                           start=line_start,
                                           end=line_end,
                                           color=colors.by_idx(i))
    
            line_zone.trigger(detections=detections_filtered)
    
        # Display vehicle counts by type 
        type_text_position = (width - 300, 30)
        vehicle_count_text = " | ".join([f"{class_names[class_id]}: {count}" for class_id, count in vehicle_type_counts.items()])
        cv2.putText(annotated_frame, f"Vehicle Types: {vehicle_count_text}", type_text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
    
        # Traffic congestion detection
        congestion_threshold = 10  # Example threshold for congestion
        if total_car_count > congestion_threshold:
            congestion_text_position = (width - 300, 60)
            cv2.putText(annotated_frame, "WARNING: Traffic Congestion Detected!", congestion_text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
            print(f"Traffic congestion detected: {total_car_count} vehicles.")
    
        # zone-based vehicle heatmap (Basic version)
        heatmap_color = (0, 0, 255) if total_car_count > congestion_threshold else (0, 255, 0)
        for i, count in enumerate(zone_car_counts):
            zone_text_position = (width - 300, 90 + i * 30)
            cv2.putText(annotated_frame, f"Zone {i+1} Cars: {count}", zone_text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.6, heatmap_color, 2)
    
        # Display total car count and warnings 
        total_text_position = (width - 300, 120 + len(zone_car_counts) * 30)
        cv2.putText(annotated_frame, f"Total Cars: {total_car_count}", total_text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
    
        if warning_message:
            warning_text_position = (width - 300, 150 + len(zone_car_counts) * 30)
            cv2.putText(annotated_frame, "WARNING: Cars too close!", warning_text_position, cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            print(warning_message)  
    
        return annotated_frame
    
    cap = cv2.VideoCapture(VIDEO)
    fps = video_info.fps
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    print(f"FPS: {fps}")
    print(f"Resolution: {width}x{height}")

    while True:
        start_time = time.time()
        ret, frame = cap.read()
        if not ret:
            break
        
        show = process_frame(frame, int(fps))
        fps_monitor.tick()
        
        # Calculate and display the actual FPS
        actual_fps = 1 / (time.time() - start_time)
        fps_text = f"FPS: {actual_fps:.2f}"
        cv2.putText(show, fps_text, (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Display the processed frame
        cv2.imshow("Counting - Speed Estimation", show)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        processing_time = time.time() - start_time
        sleep_time = max(1/fps - processing_time, 0)
        time.sleep(sleep_time)
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()